In [ ]:
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import sys
import numpy as np
#helper modules
import glutil
from vector import Vec

import clgl




class window():
    def __init__(self, X, B):
        #mouse handling for transforming scene
        self.mouse_down = False
        self.mouse_old = Vec([0., 0.])
        self.rotate = Vec([0., 0., 0.])
        self.translate = Vec([0., 0., 0.])
        self.initrans = Vec([0., 0., -100.])

        self.width = 1000
        self.height = 1000

        glutInit(sys.argv)
        glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE | GLUT_DEPTH)
        glutInitWindowSize(self.width, self.height)
        glutInitWindowPosition(0, 0)
        self.win = glutCreateWindow("")

        #gets called by GLUT every frame
        glutDisplayFunc(self.draw)

        #handle user input
        glutKeyboardFunc(self.on_key)
        glutMouseFunc(self.on_click)
        glutMotionFunc(self.on_mouse_motion)
        
        #this will call draw every 30 ms
        glutTimerFunc(30, self.timer, 30)

        #setup OpenGL scene
        self.glinit()
        self.loadVBO(X)
        #set up initial conditions
        

        
        #(pos_vbo, col_vbo, vel) = initialize.fountain(num)
        #create our OpenCL instance
        self.cle = clgl.CLGL()
        self.cle.loadData(self.pos_vbo, self.col_vbo, B)
        glutMainLoop()
        #self.draw()

        

    def glinit(self):
        glViewport(0, 0, self.width, self.height)
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluPerspective(60., self.width / float(self.height), .1, 1000.)
        glMatrixMode(GL_MODELVIEW)


    ###GL CALLBACKS
    def timer(self, t):
        glutTimerFunc(t, self.timer, t)
        glutPostRedisplay()

    def on_key(self, *args):
        ESCAPE = '\033'
        if args[0] == ESCAPE or args[0] == 'q':
            sys.exit()
        elif args[0] == 't':
            print self.cle.timings

    def on_click(self, button, state, x, y):
        if state == GLUT_DOWN:
            self.mouse_down = True
            self.button = button
        else:
            self.mouse_down = False
        self.mouse_old.x = x
        self.mouse_old.y = y

    
    def on_mouse_motion(self, x, y):
        dx = x - self.mouse_old.x
        dy = y - self.mouse_old.y
        if self.mouse_down and self.button == 0: #left button
            self.rotate.x += dy * .2
            self.rotate.y += dx * .2
        elif self.mouse_down and self.button == 2: #right button
            self.translate.z -= dy * .1 
        self.mouse_old.x = x
        self.mouse_old.y = y
    ###END GL CALLBACKS


    def draw(self):
        """Render the particles"""        
        #update or particle positions by calling the OpenCL kernel
        self.cle.execute(0.05) 
        glFlush()

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()

        #handle mouse transformations
        glTranslatef(self.initrans.x, self.initrans.y, self.initrans.z)
        glRotatef(self.rotate.x, 1, 0, 0)
        glRotatef(self.rotate.y, 0, 1, 0) #we switched around the axis so make this rotate_z
        glTranslatef(self.translate.x, self.translate.y, self.translate.z)
        
        
        #render the particles
        self.cle.render()

        #draw the x, y and z axis as lines
        #glutil.draw_axes()
        glutSwapBuffers()

        
    def loadVBO(self, X):
        self.pos = X
        
        self.col = np.zeros_like(self.pos)
        self.col[:,:,:,0] = 0.
        self.col[:,:,:,1] = 1.
        self.col[:,:,:,2] = 0.
        self.col[:,:,:,3] = 1.

        #create the Vertex Buffer Objects
        from OpenGL.arrays import vbo 
        self.pos_vbo = vbo.VBO(data=self.pos, usage=GL_DYNAMIC_DRAW, target=GL_ARRAY_BUFFER)
        self.pos_vbo.bind()
        self.col_vbo = vbo.VBO(data=self.col, usage=GL_DYNAMIC_DRAW, target=GL_ARRAY_BUFFER)
        self.col_vbo.bind()
        
        return self


if __name__ == "__main__":
    dim = (32,32,32)
    
    idx = np.indices(dim).astype(np.float32)  
    X = np.zeros(dim+(4,), dtype = np.float32)
    X[:,:,:,0] = idx[0,:,:,:]
    X[:,:,:,1] = idx[1,:,:,:]
    X[:,:,:,2] = idx[2,:,:,:]

    X1 = np.array(X)
    ar = 4**2
    az = 4**2
    h = np.exp(-((X[:,:,:,0]-dim[0]/2)**2 + (X[:,:,:,1]-dim[1]/2)**2)/ar - (X[:,:,:,2]-dim[2]/2)**2/az)*np.pi*2

    X[:,:,:,0] = (X1[:,:,:,0]-dim[0]/2)*np.cos(h) - (X1[:,:,:,1]-dim[1]/2)*np.sin(h)# + dim[0]/2
    X[:,:,:,1] = (X1[:,:,:,0]-dim[0]/2)*np.sin(h) + (X1[:,:,:,1]-dim[1]/2)*np.cos(h)# + dim[1]/2

    #X*= 0.015
    X[:,:,:,3] = 1.
    
    B = np.zeros(dim+(4,), dtype = np.float32)
    B[:,:,:,2] = np.ones(dim, dtype = np.float32)
    
    #X*=0.01
    
    #X = (np.random.random(dim+(4,)).astype(np.float32) - 0.5)*0.3
    p2 = window(X, B)

In [3]:
X

array([[[[ 0.10361183, -0.14341062,  0.06320204,  0.03369667],
         [-0.00853001,  0.04948395,  0.11840861, -0.12639709],
         [-0.12220512,  0.02089752, -0.12416761, -0.05452113],
         ..., 
         [ 0.12439124,  0.09010835,  0.12952727, -0.10004942],
         [ 0.07678399,  0.03010043, -0.01373138, -0.01344036],
         [-0.03990285, -0.09252109, -0.01599058,  0.04346414]],

        [[-0.14257309,  0.07364585, -0.03267718, -0.09034848],
         [-0.07796223, -0.0442098 ,  0.06790976, -0.01016974],
         [ 0.09183355,  0.02082104, -0.13677344,  0.06026706],
         ..., 
         [ 0.00190592, -0.13811065,  0.07066976,  0.11902457],
         [ 0.1467177 , -0.11672769,  0.09045461, -0.02601605],
         [ 0.06826439,  0.14978367, -0.12624389, -0.14866947]],

        [[-0.14765187, -0.14425801,  0.07018651,  0.03746891],
         [ 0.02931685,  0.07423611, -0.04788701,  0.0364811 ],
         [-0.12785782,  0.06438065, -0.06037135, -0.1307627 ],
         ..., 
      

In [5]:
from OpenGL.arrays import vbo 
pos_vbo = vbo.VBO(data=X, usage=GL_DYNAMIC_DRAW, target=GL_ARRAY_BUFFER)